In [1]:
from __future__ import unicode_literals, print_function, division

In [2]:
from io import open

In [3]:
import glob

In [4]:
import unicodedata

In [5]:
import string

In [6]:
all_letters = string.ascii_letters + " .,;'-"

In [7]:
n_letters = len(all_letters) + 1

In [8]:
n_letters

59

In [9]:
print(n_letters)

59


In [10]:
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'-"

In [11]:
def findFiles(path): return glob.glob(path)

In [12]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [13]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [14]:
# build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('/home/damvantai/Documents/data/data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)
print("# categories:", n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Korean', 'Russian', 'English', 'French', 'Dutch', 'Vietnamese', 'Czech', 'German', 'Chinese', 'Spanish', 'Greek', 'Arabic', 'Irish', 'Polish', 'Japanese', 'Scottish', 'Portuguese', 'Italian']
O'Neal


### Creating the network

In [15]:
import torch

In [16]:
import torch.nn as nn
from torch.autograd import Variable

In [17]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax()
        
    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

In [18]:
import random

In [19]:
def randomChoice(l):
    return l[random.randint(0, len(l) -1)]

In [20]:
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [21]:
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

In [22]:
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor


In [23]:
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1)
    return torch.LongTensor(letter_indexes)

In [24]:
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = Variable(categoryTensor(category))
    input_line_tensor = Variable(inputTensor(line))
    target_line_tensor = Variable(targetTensor(line))
    return category_tensor, input_line_tensor, target_line_tensor

In [25]:
criterion = nn.NLLLoss()
learning_rate = 0.0005

In [26]:
def train(category_tensor, input_line_tensor, target_line_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    loss = 0
    
    for i in range(input_line_tensor.size()[0]):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        loss += criterion(output, target_line_tensor[i])
        
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.data[0] / input_line_tensor.size()[0]

In [27]:
import time

In [28]:
import math

In [29]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


In [30]:
rnn = RNN(n_letters, 128, n_letters)
n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0

In [31]:
start = time.time()
for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss
    
    if iter % print_every == 0:
        print("%s (%d %d%%) %.4f" % (timeSince(start), iter, iter / n_iters * 100, loss))
        
    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0
        

0m 52s (5000 5%) 2.8990
1m 43s (10000 10%) 2.4454
2m 34s (15000 15%) 2.5485
3m 25s (20000 20%) 1.6170
4m 18s (25000 25%) 2.1843
5m 12s (30000 30%) 1.7838
6m 7s (35000 35%) 2.1945
7m 4s (40000 40%) 2.1452
8m 2s (45000 45%) 2.9520
8m 52s (50000 50%) 2.8211
9m 42s (55000 55%) 3.6016
10m 32s (60000 60%) 2.5416
11m 24s (65000 65%) 2.8750
12m 16s (70000 70%) 2.7785
13m 6s (75000 75%) 2.1384
13m 56s (80000 80%) 2.7790
14m 45s (85000 85%) 2.3199
15m 35s (90000 90%) 2.6583
16m 24s (95000 95%) 2.8571
17m 14s (100000 100%) 2.4717


In [32]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [33]:
plt.figure()
plt.plot(all_losses)

In [34]:
max_length = 20

In [35]:
# sample from a category and starting letter
def sample(category, start_letter = 'A'):
    category_tensor = Variable(categoryTensor(category))
    input = Variable(inputTensor(start_letter))
    hidden = rnn.initHidden()
    
    output_name = start_letter
    
    for i in range(max_length):
        output, hidden = rnn(category_tensor, input[0], hidden)
        topv, topi = output.data.topk(1)
        topi = topi[0][0]
        if topi == n_letters - 1:
            break
        else:
            letter = all_letters[topi]
            output_name += letter
        input = Variable(inputTensor(letter))
    
    return output_name
        
            

In [36]:
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

In [37]:
samples('Russian', 'RUS')
samples('German', 'GER')
samples('Spanish', 'SPA')
samples('Chinese', 'CHI')

Rovellovov
Uaren
Shanak
Gerten
Eeren
Roner
Sarela
Parer
Allan
Chang
Hai
Iun


In [39]:
samples('Vietnamese', 'damvantai')

dan
ara
man
van
ana
na
than
ana
ina
